In [5]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   --------------- ------------------------ 3.8/9.5 MB 80.8 MB/s eta 0:00:01
   --------------------------- ------------ 6.6/9.5 MB 70.0 MB/s eta 0:00:01
   -------------------------------------- - 9.2/9.5 MB 65.9 MB/s eta 0:00:01
   ---------------------------------------  9.5/9.5 MB 68.0 MB/s eta 0:00:01
   ---------------------------------------  9.5/9.5 MB 68.0 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 40.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/492.9 kB ? eta -:--:--
   --------------------------------------- 492.9/492.9 kB 32.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/63.8 kB ? eta -:--:--
   ---------------------------------------- 63.8/63.8 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


pip install -r requirements.txt

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import os
import time
import glob

# --- [1] 설정: 크롬 드라이버 경로 및 다운로드 폴더 ---
driver_path = r'C:\Users\박주현\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'
download_dir = r'C:\Users\박주현\Downloads'  # 크롬 다운로드 기본 경로
save_dir = r'C:\Users\박주현\Documents\UNSD total ghg data'

# --- [2] Selenium 옵션 설정 ---
options = Options()
options.headless = False  # 브라우저 보이게 설정
prefs = {'download.default_directory': download_dir}
options.add_experimental_option('prefs', prefs)

# --- [3] 웹 드라이버 실행 ---
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://di.unfccc.int/detailed_data_by_party')
time.sleep(3)

# --- [4] 첫 번째 드롭다운: 국가 목록 가져오기 ---
select1 = Select(driver.find_element(By.XPATH, '/html/body/div/div/div[2]/div/div/select[1]'))
first_dropdown_options = [option.text for option in select1.options if option.text != '--Select Party--']

# --- [5] 국가별 반복 시작 ---
for first_option in first_dropdown_options:
    print(f"\n🌐 국가 처리 중: {first_option}")
    try:
        select1.select_by_visible_text(first_option)
        time.sleep(2)

        # --- [6] 세 번째 드롭다운 옵션 확인 ---
        select3_element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div[2]/div/div/select[3]'))
        )
        select3 = Select(select3_element)
        third_options = [opt.text.strip() for opt in select3.options]

        # --- [7] 사용할 GHG 옵션 필터링 ---
        target_third_options = [
            "....Total GHG emissions excluding LULUCF/LUCF",
            "....Total GHG emissions without LULUCF"
        ]
        matched_third = [opt for opt in third_options if opt in target_third_options]

        if not matched_third:
            print("❌ 사용 가능한 GHG 옵션이 없음. 건너뜀")
            continue

        # --- [8] 세 번째 옵션 순회 ---
        for third_option in matched_third:
            print(f"➡ 세 번째 옵션: {third_option}")
            select3.select_by_visible_text(third_option)
            time.sleep(1)

            # --- [9] 나머지 드롭다운 설정 (2, 4, 5번째) ---
            Select(driver.find_element(By.XPATH, '/html/body/div/div/div[2]/div/div/select[2]')).select_by_visible_text('All years')
            Select(driver.find_element(By.XPATH, '/html/body/div/div/div[2]/div/div/select[4]')).select_by_visible_text('Aggregate GHGs')

            # 다섯 번째 옵션 자동 판단
            select5 = Select(driver.find_element(By.XPATH, '/html/body/div/div/div[2]/div/div/select[5]'))
            fifth_options = [opt.text.strip() for opt in select5.options]

            if "Tg CO₂ equivalent" in fifth_options:
                selected_unit = "Tg CO2e"
                select5.select_by_visible_text("Tg CO₂ equivalent")
            elif "Mt CO₂ equivalent" in fifth_options:
                selected_unit = "Mt CO2e"
                select5.select_by_visible_text("Mt CO₂ equivalent")
            else:
                print("❌ 단위 선택 실패")
                continue

            # --- [10] 다운로드 버튼 클릭 ---
            try:
                download_button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "dt-button") and contains(@class, "buttons-excel")]//span[text()="Export to Excel"]'))
                )
                download_button.click()
                print("⬇️ 다운로드 시도")
                time.sleep(5)
            except TimeoutException:
                print("❌ 다운로드 버튼 클릭 실패")
                continue

            # --- [11] 최신 파일 이름으로 저장 ---
            try:
                label = driver.find_element(By.CLASS_NAME, 'gray').text.strip()
                safe_name = f"{first_option}_{third_option.replace('.', '').strip()}_{selected_unit}.xlsx".replace('/', '_').replace('\\', '_')

                list_of_files = glob.glob(os.path.join(download_dir, '*.xlsx'))
                if not list_of_files:
                    print("❌ 다운로드된 파일 없음")
                    continue
                latest_file = max(list_of_files, key=os.path.getctime)

                dest_path = os.path.join(save_dir, safe_name)
                os.rename(latest_file, dest_path)
                print(f"📁 저장 완료: {dest_path}")
            except Exception as e:
                print(f"❌ 파일 저장 실패: {e}")

    except Exception as e:
        print(f"❌ 국가 처리 중 오류: {e}")
        continue

# --- [12] 마무리 ---
driver.quit()
print("\n✅ 크롤링 완료")



🌐 국가 처리 중: Annex I
➡ 세 번째 옵션: ....Total GHG emissions without LULUCF
⬇️ 다운로드 시도
📁 저장 완료: C:\Users\박주현\Documents\UNSD total ghg data\Annex I_Total GHG emissions without LULUCF_Mt CO2e.xlsx

🌐 국가 처리 중: Annex I EIT
➡ 세 번째 옵션: ....Total GHG emissions without LULUCF
⬇️ 다운로드 시도
📁 저장 완료: C:\Users\박주현\Documents\UNSD total ghg data\Annex I EIT_Total GHG emissions without LULUCF_Mt CO2e.xlsx

🌐 국가 처리 중: Annex I non-EIT
➡ 세 번째 옵션: ....Total GHG emissions without LULUCF
⬇️ 다운로드 시도
📁 저장 완료: C:\Users\박주현\Documents\UNSD total ghg data\Annex I non-EIT_Total GHG emissions without LULUCF_Mt CO2e.xlsx

🌐 국가 처리 중: Australia
➡ 세 번째 옵션: ....Total GHG emissions without LULUCF
⬇️ 다운로드 시도
📁 저장 완료: C:\Users\박주현\Documents\UNSD total ghg data\Australia_Total GHG emissions without LULUCF_Mt CO2e.xlsx

🌐 국가 처리 중: Austria
➡ 세 번째 옵션: ....Total GHG emissions without LULUCF
⬇️ 다운로드 시도
📁 저장 완료: C:\Users\박주현\Documents\UNSD total ghg data\Austria_Total GHG emissions without LULUCF_Mt CO2e.xlsx

🌐 국가 처리 중: Belarus
➡ 세 번째 

In [13]:
import os
import pandas as pd
import re
import glob

# 1. Year 칼럼을 가진 기본 데이터프레임 생성
years = list(range(1960, 2025))
main_df = pd.DataFrame({'Year': years})
main_df['Year'] = main_df['Year'].astype(str)  # 문자열 형변환

# 2. 대상 폴더 경로 설정
folder_path = r'C:\Users\박주현\Documents\GHG-Dashboard'
file_paths = glob.glob(os.path.join(folder_path, "*.xlsx"))

# 3. 국가별 데이터 추출 및 정제
country_dfs = []
for file_path in file_paths:
    try:
        df = pd.read_excel(file_path, header=None)

        # "Base year"이 3행에 포함된 열 제거
        df = df.loc[:, ~df.iloc[2].astype(str).str.contains('Base year', case=False)]

        # 필요한 행만 추출
        df_filtered = df[df.iloc[:, 0].isin([
            'Category',
            'Total GHG emissions without LULUCF',
            'Total GHG emissions excluding LULUCF/LUCF'
        ])]

        # 데이터 존재 여부 확인
        if df_filtered.shape[0] < 2:
            print(f"⚠️ 필요한 행이 부족함: {file_path}")
            continue

        # 4행 1열은 'Year', 5행 1열은 국가명으로 지정
        df_filtered.iloc[0, 0] = 'Year'
        country_name = os.path.basename(file_path).split('_')[0].strip()
        df_filtered.iloc[1, 0] = country_name

        # 전치 후 처리
        country_df = df_filtered.T
        country_df.columns = country_df.iloc[0]  # 첫 행을 컬럼명으로
        country_df = country_df.drop(index=0)    # 첫 행 제거

        # Year 값 정제 (4자리 숫자 추출)
        country_df['Year'] = country_df['Year'].apply(lambda x: ''.join(re.findall(r'\d', str(x))[:4]))
        country_df['Year'] = country_df['Year'].astype(str)

        country_dfs.append(country_df)

    except Exception as e:
        print(f"❌ 처리 오류: {file_path} - {e}")

# 4. 병합
final_df = main_df.copy()
for country_df in country_dfs:
    final_df = pd.merge(final_df, country_df, on='Year', how='left')

# 5. 저장
output_path = os.path.join(folder_path, "UNFCCC_GHG_merged.xlsx")
final_df.to_excel(output_path, index=False)
print(f"✅ 병합 파일 저장 완료: {output_path}")

✅ 병합 파일 저장 완료: C:\Users\박주현\Documents\GHG-Dashboard\UNFCCC_GHG_merged.xlsx
